<a href="https://colab.research.google.com/github/jamesodukoya/BitcoinPricePrediction/blob/main/Bitcoin_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Bitcoin Sentiment Analysis Based on Wiki Entries

##Installing Wikipedia Package

In [ ]:
!pip install mwclient

##Downloading Bitcoin Wikipedia Page Edits

In [ ]:
import mwclient
import time

site=mwclient.Site('en.wikipedia.org')
page=site.pages["Bitcoin"]

In [ ]:
revs=list(page.revisions())

In [ ]:
revs[0]

In [ ]:
revs=sorted(revs, key=lambda rev: rev['timestamp'])

##Determining the Sentiment of Each Edit Using Transformers Package

In [ ]:
from transformers import pipeline

In [ ]:
sentiment_pipeline=pipeline("sentiment-analysis")

def find_sentiment(text):
  sent=sentiment_pipeline([text[:250]])[0]
  score=sent['score']
  if sent['label']=='NEGATIVE':
    score*=-1
  return score

###Calculate Total Number of Edits Per Day

In [ ]:
edits = {}

for rev in revs:
    date = time.strftime("%Y-%m-%d", rev["timestamp"])
    if date not in edits:
        edits[date] = dict(sentiments=list(), edit_count=0)

    edits[date]["edit_count"] += 1

    comment = rev.get("comment", "")
    edits[date]["sentiments"].append(find_sentiment(comment))

###Determine the Average Sentiment Score Per Day

In [ ]:
from statistics import mean

for key in edits:
    if len(edits[key]["sentiments"]) > 0:
        edits[key]["sentiment"] = mean(edits[key]["sentiments"])
        edits[key]["neg_sentiment"] = len([s for s in edits[key]["sentiments"] if s < 0]) / len(edits[key]["sentiments"])
    else:
        edits[key]["sentiment"] = 0
        edits[key]["neg_sentiment"] = 0

    del edits[key]["sentiments"]

###Pass Results into a Pandas Dataframe

In [ ]:
import pandas as pd

edits_df = pd.DataFrame.from_dict(edits, orient="index")

edits_df

In [ ]:
edits_df.index = pd.to_datetime(edits_df.index)

###Filling in Zero Values for Days Without Edits

In [ ]:
from datetime import datetime

dates = pd.date_range(start="2009-03-08", end=datetime.today())
edits_df = edits_df.reindex(dates, fill_value=0)

In [ ]:
edits_df

In [ ]:
rolling_edits = edits_df.rolling(30, min_periods=30).mean()
rolling_edits = rolling_edits.dropna()
rolling_edits

In [ ]:
rolling_edits.to_csv("wikipedia_edits.csv")